In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from itertools import islice

from torch import nn
from torch.nn import functional as F

import sys 
import getpass

sys.path.append('../')
from Miniproject_2.model import *
torch.set_grad_enabled(True);

In [2]:
def test_init_weights(model):
    if isinstance(model,nn.Conv2d) or isinstance(model,nn.ConvTranspose2d):
        model.weight.data.normal_(0,0.5, generator=torch.manual_seed(0))
        
def compare(x,y,decimals=7):
    return torch.all(torch.round(torch.abs(x - y), decimals=decimals)==0.).item()

In [3]:
train_in, train_tg = torch.load('../train_data.pkl')
train_in = train_in[:10].float()/255.
train_tg = train_tg[:10].float()/255.

val_in, val_tg = torch.load('../val_data.pkl')
val_in = val_in.float()/255.
val_tg = val_tg.float()/255.

In [4]:
stride = ks = 2

my_conv1 = Conv2d(in_channels=3, out_channels=5, stride=stride,  kernel_size=ks)
my_conv2 = Conv2d(in_channels=5, out_channels=5, stride=stride,  kernel_size=ks)


my_tconv1 = TransposeConv2d(in_channels=5, out_channels=5,  kernel_size=ks, stride=stride, padding=0, dilation=1)
my_tconv2 = TransposeConv2d(in_channels=5, out_channels=3,  kernel_size=ks, stride=stride, padding=0, dilation=1)

my_relu = ReLU()
my_sigmoid = Sigmoid()

my_Net = Sequential(my_conv1, my_relu, my_conv2, my_relu, my_tconv1, my_relu, my_tconv2, my_sigmoid)

loss = MSE()
dL_dy = loss.forward_and_vjp(train_tg)

my_Net.forward_and_vjp(train_in, dL_dy)

In [5]:
my_gradients  = []
my_parameters = []
for i, module in enumerate(my_Net._modules.values()):
    if len(module.d_weight):
        my_gradients.append(module.d_weight)
        my_parameters.append(module.weight)

In [6]:
conv1 = nn.Conv2d(in_channels=3, out_channels=5,  kernel_size=ks, stride=stride, bias=False)
conv2 = nn.Conv2d(in_channels=5, out_channels=5,  kernel_size=ks, stride=stride, bias=False)

tconv1 = nn.ConvTranspose2d(in_channels=5, out_channels=5,  kernel_size=ks, stride=stride, padding=0, dilation=1, bias=False)
tconv2 = nn.ConvTranspose2d(in_channels=5, out_channels=3,  kernel_size=ks, stride=stride, padding=0, dilation=1, bias=False)

relu = nn.ReLU()
sigmoid = nn.Sigmoid()

Net = nn.Sequential(conv1, relu, conv2, relu, tconv1, relu, tconv2, sigmoid)
Net.apply(test_init_weights);

In [7]:
valid_input, valid_target = torch.load('../val_data.pkl')
criterion = nn.MSELoss()

y = Net(train_in)
loss = criterion(y, train_tg)
loss.backward()

gradients = []
with torch.no_grad():
    for p in Net.parameters(): gradients.append(p.grad)
        
parameters = [list(Net.parameters())[i] for i in range(4)]

In [8]:
[torch.all(parameters[0]==my_parameters[0]).item() for i in range(4)]

[True, True, True, True]

In [9]:
[compare(gradients[0], my_gradients[0], decimals=4) for i in range(4)]

[True, True, True, True]